In [ ]:
import numpy as np
import math
import cv2

capture = cv2.VideoCapture('/ped_count.mov')

fgbg = cv2.bgsegm.createBackgroundSubtractorMOG(history=150, backgroundRatio=0.3)

def first_line(x,y):
    return y - (30*x)/100.0 - 300

def second_line(x,y):
    return y - (30*x)/100.0 - 500


up_to, up_low = 0,0
points,pointFromAbove,pointFromBelow = set(),set(),set()

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('pedestrianOutput.avi',fourcc, 25.0, (986,810))
font = cv2.FONT_HERSHEY_SIMPLEX
while(True):
    pointInMiddle = set()
    prev = points
    points = set()
    ret, frame = capture.read()
    fgmask = frame
    fgmask = cv2.blur(frame, (10,10))
    fgmask = fgbg.apply(fgmask)
    fgmask = cv2.medianBlur(fgmask, 7)
    oldFgmask = fgmask.copy()
    contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_EXTERNAL,1)
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        if w>40 and h>90:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2, lineType=cv2.LINE_AA)
            point = (int(x+w/2.0), int(y+h/2.0))
            points.add(point)
    for point in points:
        (xnew, ynew) = point
        if first_line(xnew, ynew) > 0 and second_line(xnew, ynew) < 0:
            pointInMiddle.add(point)
        for prevPoint in prev:
            (xold, yold) = prevPoint
            dist = cv2.sqrt((xnew-xold)*(xnew-xold)+(ynew-yold)*(ynew-yold))
            if dist[0] <= 120:
                if first_line(xnew, ynew) >= 0 and second_line(xnew, ynew) <= 0:
                    if first_line(xold, yold) < 0: 
                        pointFromAbove.add(point)
                    elif second_line(xold, yold) > 0: 
                        pointFromBelow.add(point)
                    else:   
                        if prevPoint in pointFromBelow:
                            pointFromBelow.remove(prevPoint)
                            pointFromBelow.add(point)

                        elif prevPoint in pointFromAbove:
                            pointFromAbove.remove(prevPoint)
                            pointFromAbove.add(point)

                if first_line(xnew, ynew) < 0 and prevPoint in pointFromBelow: 
                    up_to += 1
                    pointFromBelow.remove(prevPoint)

                if second_line(xnew, ynew) > 0 and prevPoint in pointFromAbove: 
                    up_below += 1
                    pointFromAbove.remove(prevPoint)


    for point in points:
        if point in pointFromBelow:
            cv2.circle(frame, point, 3, (255,0,255),6)
        elif point in pointFromAbove:
            cv2.circle(frame, point, 3, (0,255,255),6)
        else:
            cv2.circle(frame, point, 3, (0,0,255),6)
    cv2.line(frame, (0,300), (1920,880), (255, 0, 0), 4)
    cv2.line(frame, (0,500), (1920,1080), (255, 0, 0), 4)
    cv2.putText(frame,'Above = '+str(up_to),(570,40), font, 1,(0,0,255),2,cv2.LINE_AA)
    cv2.putText(frame,'Below = '+str(up_low),(570,240), font, 1,(0,0,255),2,cv2.LINE_AA)
    cv2.imshow('a',oldFgmask)
    cv2.imshow('frame',frame)
    out.write(frame)
    l = cv2.waitKey(1) & 0xff
    if l == 77:
        break
capture.release()
cv2.destroyAllWindows()